<a href="https://colab.research.google.com/github/illusion173/CS555UAV/blob/main/CS555Proj.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Necessary Commands to run before import
%pip install ultralytics

In [2]:
# Import needed forms
import os
import collections
import pandas as pd
import numpy as np
import functools
import matplotlib.pyplot as plt
import cv2
from PIL import Image
import tensorflow as tf
from sklearn.model_selection import train_test_split
import warnings
import zipfile
from google.colab import drive
import ultralytics


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


**GOOGLE DRIVE MOUNTING/UNZIPPING OF DATASET**

In [3]:
# DO NOT TOUCH PLEASE, loc of your zip file should match this
# the zip should be in the root of your google drive.

GOOGLE_DRIVE_DATA_DIR_PATH = "/content/drive/"
drive.mount(GOOGLE_DRIVE_DATA_DIR_PATH, force_remount=True)
%cd /content/drive/MyDrive/
# Unzip the file into /tmp using -d, shouldn't take too long <1 min
!unzip -q "uav_datasets.zip" -d /tmp


Mounted at /content/drive/
/content/drive/MyDrive
replace /tmp/dataset_xml_format/dataset_xml_format/foto00088.png? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

**DATA INGESTION/ORGANIZATION**

In [6]:
# KEEP NOTE THAT THE DIRECTORIES MADE IN THIS CELL ARE IN THE /tmp dir and WILL BE DELETED
YOLO_DATASET_FOLDER = "/tmp/drone_dataset_yolo/dataset" # This dir made in the above unzip cell
TRAIN_IMAGES_FOLDER = "/tmp/train/images"
TRAIN_LABELS_FOLDER = "/tmp/train/labels"
VAL_IMAGES_FOLDER = "/tmp/val/images"
VAL_LABELS_FOLDER = "/tmp/val/labels"
os.makedirs(TRAIN_IMAGES_FOLDER, exist_ok=True)
os.makedirs(TRAIN_LABELS_FOLDER, exist_ok=True)
os.makedirs(VAL_IMAGES_FOLDER, exist_ok=True)
os.makedirs(VAL_LABELS_FOLDER, exist_ok=True)

In [7]:
# Creating necessary lists for organizing the files
images = []
labels = []


In [8]:
# Assigning necessary split for the training/testing 80/20
